## <font color='blue'>SME0104 - Cálculo Numérico</font>

### Primeiro Trabalho em Grupo 
#### Comparação de Métodos na Solução do Laplaciano em Grafos para propagação de informação

**Luis Gustavo Nonato**

------

Considere os arquivos `manh.el` e `manh.xy` que fornecem as arestas e as coordenadas dos vértices do grafo de ruas da ilha de Manhattan, NY (arquivos disponíveis para download no Google Drive).

O grafo de ruas possui diversas componentes conexas, considerando somente a maior componente conexa, você deve realizar as seguintes tarefas:
1. Selecione alguns vértices do grafo $v_{i_1},v_{i_2},\ldots,v_{i_k},\, k<<n$ ($n$ é o número de vértices na maior componente do grafo e $k$ é um número bem menor que $n$, $k=10$ por exemplo) e atribua valores distindos $c_{i_1},c_{i_2},\ldots,c_{i_k}$ a cada um dos vértices selecionados (por exemplo valores no intervalo (0,10]);

---


2. Construa a matriz Laplaciana $L$ do grafo de ruas;
3. Construa a matriz de penalidades $P$, sendo $P$ é uma matriz diagonal onde a entrada $P_{jj}=\alpha$ se $j$ corresponde ao índice de algum dos vértices escolhidos no item 1 acima ($\alpha=1.0e7$ por exemplo), sendo $P_{ii}=0$ caso contrário.
4. Construa um vetor $b$ da seguinte forma:
$$
b_{j} = \left\{\begin{matrix} c_{i_s} & \:{ se } \: j = i_s \\ 0 & \:{ caso \: contrário}\end{matrix}\right.
$$
5. Compare o tempo de solução do sistema
$$
(L+P)x = b
$$
para os métodos:
    - Decomposição LU
    - Cholesky
    - Jacobi e Gaus-Seidel
    - Gradientes Conjugados
    
6. Refaça as tarefas com representação por matriz esparsa e matrizes cheias, comparando os resultados.

Alunos: 
- Kauê Hunnicutt Bazilli - 11212226
- Matheus Vieira Gonçalves - 11200397
- Pedro Henrique dias Junqueira de Souza - 11294312

In [14]:
import sys
import time
%pip install --prefix {sys.prefix} numpy
%pip install --prefix {sys.prefix} scipy
%pip install --prefix {sys.prefix} networkx
%pip install --prefix {sys.prefix} matplotlib

import numpy as np
import scipy as sci
from scipy.sparse import linalg
import networkx as nx
import matplotlib.pyplot as plt
from matplotlib import cm

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


In [15]:
# Construção do grafo em memória com base nos arquivos
G = nx.Graph()

#Nós
with open("./manh.xy","r") as file:
    lines = file.readlines()
    for i, line in enumerate(lines):
        [x, y] = map(float, line.split("\t")) #Lê como floats
        G.add_node(i, data=(x, y))

#Arestas
with open("./manh.el","r") as file:
    lines = file.readlines()
    for line in lines:
        [x, y] = map(int, line.split("\t")) #Lê como ints
        G.add_edge(x,y)
        
LCC_generator = max(nx.connected_components(G), key=len)
G = G.subgraph(LCC_generator).copy()

# Desse ponto em diante, G é a maior componente conexa

In [16]:
# 1) Selecione alguns vértices do grafo e atribua valores distintos a cada um dos vértices selecionados

# Uma lista dos índices e outra lista com os seus respectivos valores
c_indexes = np.random.randint(0, len(G), 10)
c_values = np.random.uniform(0, 10, len(c_indexes))
print(c_indexes)

[8226 1129 5843 2367 5188 3668 8565 4302  109 1424]


In [17]:
# 2) Construa a matriz Laplaciana L do grafo de ruas

# Obtendo a matriz laplaciana
L = np.array(nx.laplacian_matrix(G).todense())
L.shape

(8708, 8708)

In [18]:
# 3) Construa a matriz de penalidades P

P_diag =  [1e7 if i in c_indexes else 0 for i in range(len(G))]
P = np.diag(P_diag)

In [19]:
# 4) Construa o vetor b 
# Px = b
# b é um vetor com ci nos índices i escolhidos na 1) e 0 em todo o resto.
b = np.zeros(len(G))
for i,ci in enumerate(c_indexes):
    b[ci] = c_values[i]

In [42]:
# TODO: 5) Compare o tempo de solução do sistema (L + P)x = Px = b para os métodos:
#   - Decomposição LU
#   - Cholesky
#   - Jacobi e Gaus-Seidel
#   - Gradientes Conjugados
# NOTE: Pode usar métodos embutidos do numpy e etc...
#
# Lx = 0
# Px = b
#
# Nem P nem L são invertíveis, mas (L + P) é, então resolvemos a soma dos sistemas:
#
# (L + P)x = Px = b

L_plus_P_dense = np.add(L, P)
L_plus_P_csc_sparse =sci.sparse.csc_matrix(L_plus_P_dense)

b_dense = b

print(L_plus_P_csc_sparse.shape)

print(L_plus_P_dense.shape)

print(b_dense.shape)


(8708, 8708)
(8708, 8708)
(8708,)


In [40]:
# Decomposição LU

def dense_lu_solver(A,b):
	lu, piv = sci.linalg.lu_factor(A)
	x = sci.linalg.lu_solve((lu,piv),b)
	return x

def sparse_lu_solver(A,b):
	lu = linalg.splu(A)
	x = lu.solve(b)
	return x


In [ ]:
# Decomposição de cholesky

def dense_cholesky_solver(A,b):
	c, low = sci.linalg.cho_factor(A)
	x = sci.linalg.cho_solve((c,low),b)
	return x

def sparse_cholesky_solver(A,b):
	c, low = sci.linalg.cho_factor(A)
	x = sci.linalg.cho_solve((c,low),b)
	return False


In [ ]:
# gradientes conjugados

def sparse_cg_solver(A, b):
    return linalg.cg(A, b, tol=1e-12)


def dense_cg_solver(A, b):
    return linalg.cg(A, b, tol=1e-12)



In [ ]:
# Gauss-jacobi

def dense_jacobi_solver(A, b, err, err_relative):
    count = 0
    r, c = A.shape
    x = np.zeros(r)

    D = np.diag(A)
    R = A - np.diagflat(D)
    curr_err = err + 1
    curr_err_relative = err_relative + 1

    while curr_err > err or curr_err_relative > err_relative:
        last_x = x.copy()
        x = (b - np.dot(R, x)) / D

        max_distance = -1
        max_module = -1

        for i in range(c):
            curr_distance = abs(last_x[i] - x[i])
            max_distance = curr_distance if curr_distance > max_distance else max_distance
            max_module = abs(x[i]) if abs(x[i]) > max_module else max_module

        curr_err = max_distance
        curr_err_relative = max_distance/max_module

    return x




In [ ]:
# Gauss-Seidel
def dense_gauss_seidel_solver(A, b, err, err_relative):
    r, c = A.shape
    x = np.zeros(r)
    count = 0
    curr_err = err + 1
    curr_err_relative = err_relative + 1

    while curr_err > err or curr_err_relative > err_relative:

        last_x = x.copy()

        for i in range(r):
            x[i] = (b[i] - np.dot(A[i, :i], x[:i]) -
                    np.dot(A[i, (i+1):], last_x[(i+1):])) / A[i, i]

        max_distance = -1
        max_module = -1

        for i in range(c):
            curr_distance = abs(last_x[i] - x[i])
            max_distance = curr_distance if curr_distance > max_distance else max_distance
            max_module = abs(x[i]) if abs(x[i]) > max_module else max_module

        curr_err = max_distance
        curr_err_relative = max_distance/max_module

    return x



In [ ]:
def execute_method(method, *args):
	start = time.process_time()
	return method(*args), time.process_time() - start

In [ ]:
std_dense_solution, std_dense_solution_time = execute_method(
    np.linalg.solve, L_plus_P_dense, b_dense)

print(std_dense_solution)
print(f"tempo do método padrão np.linalg.solve: {std_dense_solution_time}")

In [ ]:
std_sparse_solution, std_sparse_solution_time = execute_method(
    linalg.spsolve, L_plus_P_csc_sparse, b_dense)

print(std_sparse_solution)
print(
    f"tempo do método padrão scipy.sparse.linalg.solve: {std_sparse_solution_time}")

In [ ]:
dense_lu_solution, dense_lu_solution_time = execute_method(
    dense_lu_solver, L_plus_P_dense, b_dense)

print(dense_lu_solution)
print(
    f"tempo do método de decomposição LU em uma matriz densa: {dense_lu_solution_time}")

In [ ]:

sparse_lu_solution, sparse_lu_solution_time = execute_method(
    sparse_lu_solver, L_plus_P_csc_sparse, b_dense)

print(sparse_lu_solution)
print(
    f"tempo do método de decomposição LU em uma matriz esparsa: {sparse_lu_solution_time}")

In [ ]:
dense_cholesky_solution, dense_cholesky_solution_time = execute_method(
    dense_cholesky_solver, L_plus_P_dense, b_dense)

print(dense_cholesky_solution)
print(
    f"tempo do método de decomposição cholesky em uma matriz densa: {dense_cholesky_solution_time}")

In [ ]:

dense_cg_solution, dense_cg_solution_time = execute_method(
    dense_cg_solver, L_plus_P_dense, b_dense)

print(dense_cg_solution)
print(
    f"tempo do método de gradiente conjugados em uma matriz densa: {dense_cg_solution_time}")

In [ ]:
sparse_cg_solution, sparse_cg_solution_time = execute_method(
    sparse_cg_solver, L_plus_P_csc_sparse, b_dense)

print(sparse_cg_solution)
print(
    f"tempo do método de gradiente conjugados em uma matriz esparsa: {sparse_cg_solution_time}")

In [ ]:
dense_jacobi_solution, dense_jacobi_solution_time = execute_method(
    dense_jacobi_solver, L_plus_P_dense, b_dense, 1e-12, 1e-12)

print(dense_jacobi_solution)
print(
    f"tempo do método de Jacobi em uma matriz densa: {dense_jacobi_solution_time}")

In [ ]:
dense_gauss_seidel_solution, dense_gauss_seidel_solution_time = dense_gauss_seidel_solver(
    L_plus_P_dense, b_dense, 1e-12, 1e-12)

print(dense_gauss_seidel_solution)
print(
    f"tempo do método de Jacobi em uma matriz densa: {dense_gauss_seidel_solution_time}")